In [1]:
## libraries
import pandas as pd
import json
import re
import warnings
import pickle
import jgraph
import collections
import time
from IPython.display import Markdown
warnings.filterwarnings('ignore')


## Political Discourse Network  

#### 1. Claim clusters and cleaned actor list 

In [2]:
with open("clustered_claims_PolDis_v2_kmeansA.pkl", 'rb') as f:
    ##                                  max_df,min_df,ngram_range
    ## claim_list,top6_cluster_features,0.2,   0.02,  (1,6)
    arr_text = pickle.load(f)
    claim_text = arr_text[0]
    claim_names = arr_text[1]

with open("clean_actors_PolDis_v2.pkl", 'rb') as f:
    actor = pickle.load(f)
with open("supp_opp_ank.pkl", 'rb') as f:
    supp_opp = pickle.load(f)

#### 2. Timestamp list for time-lapse graph 

In [3]:
## gather trimestamps
claim_text_ank = open("Claims_ank.txt").read()
claims_split = claim_text_ank.split("\n\n#") 
# actor = []
# claims = []
timestamps = []
count = 0
for c in claims_split:
    txt = c.split("DATED: ")[-1]
    txt1 = txt.split("/")[0]+"/"+txt.split("/")[1]
#     txt = txt.replace(txt1,"")
    timestamps.append(txt1)
    count+=1

In [4]:
## for finding which actors are pronouns
pronoun_index = []
act_count = 0
for i in claim_text['actor']:
    if(i.lower() == "'i'" or i.lower() == "'he'"):
        pronoun_index.append(act_count)
    act_count+=1

In [5]:
claim_text['actor']
print(pronoun_index)
# for i in pronoun_index:
#     print("#######################")
#     print(claim_text['actor'][i])
#     print(claim_text['claims'][i])

[3, 44, 57, 63, 81, 85, 90, 99, 135, 140, 146, 241, 281]


In [6]:
## code for colour print display
# display (Markdown('this is in <span style="color:#ff0000">'+act_claim+'</span> color.'))

#### Actor clustering 

In [7]:
## special curated list
act_stopwords = ['mr.','dr.','i','the','a','we','of','and','other ','at','his','number','some',
             'officials','department','programs','agency','coalition','leaders','organizations','expert'
            'groups','powerful','organization','institute','trade', 'industry','advisers','laboratories',
             'fund','policy','university','panel','treaty','union','view','dominant','senator']
tricky_words = ['president','administrations','countries']

## actor group clustering
act_count = 0
act_cluster = [-1]*len(actor)
cluster_no = 0

## regex-substituter for actor stopwords
replace_stopwords = re.compile('|'.join(map(re.escape, act_stopwords)))
act_cluster_keys = []
## searching for word by word exactness of actors
while True: ######################################################### SUPER_LOOP START
    ## stop words replaced, so checking not necessary
    a1 = (replace_stopwords.sub("",actor[act_count].lower())).split()       # [actor-1] to be checked with....
    sub_act_count = 0
    ## every super-loop restarts [actor-1] as a NEW CLUSTER
    act_cluster[act_count]=cluster_no
    
    ## looping through every actor till a cluster is formed
    while True:  ##------------------------------------------## sub_LOOP START
        ## stop words replaced, so checking not necessary
        a2 = (replace_stopwords.sub("",actor[sub_act_count].lower())).split()   # ...[actor-2] for semantic overlap

        ## proceeding only if [actor-2] is NOT CLUSTERED
        if(act_cluster[sub_act_count]==-1):                
                ## adding actors to clusters based on word exactness
                
                ## need two matches minimum per actor for actors with tricky words
                a_match = 1+ int(any(i in a1 for i in tricky_words) or any(i in a2 for i in tricky_words))
                                
                ## 1. checking if they are an exact match
                if("".join(a1) == "".join(a2)):         
                    act_cluster[sub_act_count] = cluster_no

                ## 2. checking for partial match except 'stopwords'
                else:
                    ## looping actor-1 with actor-2 for partial match
                    for i in a1:   #-----------------------------------------## actor-1 START
                        for j in a2:         #-----------------------------------------## actor-2 START
                            ## word by word exactness
                            if(i == j):
                                a_match-=1
                            ##baby-loop-1--------------------------------------## actor-2 END        
                        ##baby-loop-1--------------------------------------## actor-1 END
                        if(a_match<=0):
                            act_cluster[sub_act_count] = cluster_no
                            break
                                
        ## sub-loop count iterator ---------------------------## sub_LOOP END
        sub_act_count+=1
        ## sub-loop maker and breaker
        try:
            temp = actor[sub_act_count]
        except:
            break            
        while act_cluster[sub_act_count]!=-1:
            sub_act_count+=1
            if(sub_act_count==(len(actor)-1)):
                break
            

    act_cluster_keys.append(actor[act_count])        
    ## super-loop count iterator##################################### SUPER_LOOP END
    act_count+=1
    ## super-loop maker and breaker
    try:
        temp = actor[act_count]
    except:
        break
    while act_cluster[act_count]!=-1:
        act_count+=1
        if(act_count==(len(actor)-1)):
            break
    ## at every super loop, one cluster is formed
    cluster_no+=1
                    

In [8]:
## the colour pallete for the network
##          dark    -  colours     
claim_color_pallete = [0x490357 ,0x022055 ,0x194B00 ,0x410200 ,0x512402  ]
##                     black     increment
actor_color_pallete = [0xaaaaaa ,0x010a71  ]

In [9]:
## kMeans cluster keys
claim_text.keys()
# act_cluster

dict_keys(['actor', 'claims', 'cluster'])

In [10]:
## cluster counts
# act_cluster_counts = {str(i):act_cluster.count(i) for i in act_cluster}
# print(len(act_cluster_counts))
# print()
# claim_cluster_counts = {str(i):claim_text['cluster'].count(i) for i in claim_text['cluster']}
# print(len(claim_cluster_counts))
timestamps_counts = {str(i):timestamps.count(i) for i in timestamps}
print(timestamps_counts)

{'1995/01': 10, '1997/01': 1, '1997/02': 4, '1997/03': 11, '1997/06': 84, '1997/07': 23, '1997/08': 26, '1997/09': 36, '1997/10': 91}


In [11]:
timestamps_counts.keys()

dict_keys(['1995/01', '1997/01', '1997/02', '1997/03', '1997/06', '1997/07', '1997/08', '1997/09', '1997/10'])

In [12]:
## discourse newtork graph
pol_dis_nw     =  {
                    'nodes':{},
                    'edges':{}
                    }
edge_subdict    = {'source': '', 'target': '', 'size': 0, 'color':0x000000}
node_subsubdict = {'color': 0x000000, 'size': 0.0}
claim_text['supp_opp'] = supp_opp 

#### Political Discourse graph of all actor and claim clusters

In [13]:
### for making the actor claim discourse network
def act_claim_graph(claim_text,act_cluster,actor,supp_opp,init,end):
    ## main vars
    act_cluster = act_cluster[init:end]
    actor = actor[init:end]
    cluster = claim_text['cluster'][init:end]
    supp_opp = supp_opp[init:end]
    
    ## cluster counts
    act_cluster_counts = {str(i):act_cluster.count(i) for i in act_cluster}
    claim_cluster_counts = {str(i):cluster.count(i) for i in cluster}


    ## the colour pallete for the network
    ##          dark    -  colours     
    claim_color_pallete = [0x490357 ,0x022055 ,0x194B00 ,0x410200 ,0x512402  ]
    ##                     colour    + increment
    actor_color_pallete = [0xaaaaaa ,0x010a71  ]
    
    ## -1. timewise-graph creator

    ## 0. the jgraph 
    pol_dis_nw     =  {
                    'nodes':{},
                    'edges':{}
                    }
    ## 1. edge propertoes
    edge_size = {'+':1.5,'-':0.5}
    edge_col = {'+':0xFFAF03,'-':0xaaaaaa}
    
    ## 2. nodes
    ## a. actor nodes
    nodes = {}
    node_colours = {}
    
    act_col = actor_color_pallete[0]
    actXsize = (1/max(act_cluster_counts.values()))*1 
    for key in act_cluster_counts.keys():
        i = act_cluster_counts[key]
        i = 0.5+(actXsize*i)                        ## actor node size range(0.5,1.5)
        ## storing node clour for index
        node_colours[key] = act_col
        sub_node = {'color': act_col,'size':i}
        nodes[key] = sub_node
        act_col+=actor_color_pallete[1]
    #     break
    ## b. claim nodes
    # for colour, 'claim_color_pallete'
    claimXsize = (1/max(claim_cluster_counts.values()))*2.5
    for key in claim_cluster_counts.keys():
        i = claim_cluster_counts[key]
        i = 1+(claimXsize*i)                           ## claim node size range(1,3.5)
        ## storing node clour for index
        node_colours['C'+key] = claim_color_pallete[int(key)]
        sub_node = {'color': claim_color_pallete[int(key)],'size':i}
        nodes['C'+key] = sub_node
        act_col+=actor_color_pallete[1]
    #     break
    pol_dis_nw['nodes'] = nodes
    
    
    ## 3. edges
    ## a. actor nodes
    edges = []
    for i in range(0,len(actor)):
        source = act_cluster[i]
        e_size = edge_size[supp_opp[i]]
        e_col = edge_col[supp_opp[i]]
        sub_edge = {'source': str(source), 'target': 'C'+str(cluster[i]), 'size': 1, 'color': e_col}
        edges.append(sub_edge)
    #     break
    pol_dis_nw['edges'] = edges
    
    # import json
    # print( json.dumps(pol_dis_nw))
    # edge_subdict
    return(pol_dis_nw,node_colours)

#### 1. Full Graph of entire network 

In [14]:
# jgraph.draw(act_claim_graph(claim_text,act_cluster,actor,supp_opp,0,-1))

#### 2. Month-wise timelapse of actor-claim network  

In [15]:
#############################
## Custer Representatives:
    # claim_names
    # act_cluster_keys
#############################
init = 0
end = -1
for key in timestamps_counts.keys():
    end += (timestamps_counts[key])
    nw, col_ix = act_claim_graph(claim_text,act_cluster,actor,supp_opp,init,end)
    claim_start = 0
    ## date printing
    display (Markdown('<span style="color:#000000">'+key+'</span>'))
    ## index printing
    display (Markdown('<span style="color:#000000">INDEX:</span>'))
    ## actors
    display (Markdown('<span style="color:#000000">1. ACTOR NODES </span>'))
    for i in range(0,len(col_ix.keys())):
        k = list(col_ix.keys())[i]
        colours = hex(col_ix[k])[2:]
        ## actors
        if(k[0]!='C'):
            display (Markdown('Cluster_'+k+': <span style="color:#'+colours+'">'+act_cluster_keys[int(k)]+":"+'</span>'))
        else:
            break
    ## claims
    display (Markdown('<span style="color:#000000">2. CLAIM NODES </span>'))            
    for j in range(i,len(col_ix.keys())):
        k = list(col_ix.keys())[j]
        colours = hex(col_ix[k])[2:]
        cluster_name = "-".join(claim_names[int(k[1:])])
        display (Markdown('Cluster_'+k+': <span style="color:#'+colours+'">'+cluster_name+":"+'</span>'))

    jgraph.draw(nw)
#     break
#     time.sleep(15)
    init = end

<span style="color:#000000">1995/01</span>

<span style="color:#000000">INDEX:</span>

<span style="color:#000000">1. ACTOR NODES </span>

Cluster_0: <span style="color:#aaaaaa">'Shigejiro Sugiyama':</span>

Cluster_1: <span style="color:#abb51b">'I':</span>

...

Cluster_C2: <span style="color:#194b00">'s-percent-greenhouse-nations-gases-reduce:</span>

Cluster_C0: <span style="color:#490357">greenhouse-american-reduce-energy-administration-binding:</span>

Cluster_C3: <span style="color:#410200">warming-global-global-threat-gases-dioxide:</span>

Cluster_C1: <span style="color:#22055">climate-change-climate-human-scientific-''the:</span>

Cluster_C4: <span style="color:#512402">country-developing-developing-controlling-industrial-treaty:</span>